# Project 4 - WEST NILE VIRUS

## Problem Statement

West Nile virus can cause a fatal neurological disease in humans. It is mainly transmitted to people through the bites of infected mosquitoes. Treatments often involve hospitalization, intravenous fluids, respiratory support, and prevention of secondary infections. 

No vaccine is available for humans. 

Studies estimated the economic impact of the WNV disease outbreak in 2002 in Louisiana, which resulted in 24 deaths. Total epidemic costs were about 20.14 million for the 329 cases, including 9.2 million for mosquito control and public health agency costs. As mosquito control is an expensive exercise, the state would like our team to propose a cost-effective plan for pesticide deployment.

## Executive Summary

If you want to, it's great to use relative links to direct your audience to various sections of a notebook. **HERE'S A DEMONSTRATION WITH THE CURRENT SECTION HEADERS**:

### Contents: (TOBE UPDATED)
- [2017 Data Import & Cleaning](#Data-Import-and-Cleaning)
- [2018 Data Import and Cleaning](#2018-Data-Import-and-Cleaning)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [Data Visualization](#Visualize-the-data)
- [Descriptive and Inferential Statistics](#Descriptive-and-Inferential-Statistics)
- [Outside Research](#Outside-Research)
- [Conclusions and Recommendations](#Conclusions-and-Recommendations)

**If you combine your problem statement, executive summary, data dictionary, and conclusions/recommendations, you have an amazing README.md file that quickly aligns your audience to the contents of your project.** Don't forget to cite your data sources!

*All libraries used should be added here*

In [1]:
# import libraries
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.image import imread

sns.set(font_scale=1.5)
%matplotlib inline

## Data Import and Cleaning

In [2]:
# import data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
weather = pd.read_csv('../data/weather.csv')
spray = pd.read_csv('../data/spray.csv')

### TRAIN & TEST DATA

In [3]:
# delete variables that are not useful
# address related are deleted as location is determined by Latitude and Longuitude
# checked online all species are relevant for WNV
def del_var(data):
    try:
        del data['Address']
        del data['Block']
        del data['Street']
        del data['AddressNumberAndStreet']
        del data['AddressAccuracy']
        del data['Species']
    except:
        pass

In [4]:
del_var(train)

In [5]:
del_var(test)

In [6]:
# combine test results for traps with multiple rows in any single day, since we are ignoring species
import datetime
def aggregate_train(data):
    grouped = data.groupby(['Date', 'Trap', 'Latitude', 'Longitude'])
    aggregated = pd.DataFrame(grouped.agg({'NumMosquitos': np.sum, 'WnvPresent': np.max})).reset_index()
    aggregated.sort_values(by='NumMosquitos', ascending = False)
    aggregated.sort_values(by='Date', inplace=True)
    aggregated = aggregated.reset_index(drop = True)
    aggregated['Date'] = pd.to_datetime(aggregated.Date, format='%Y-%m-%d')
    aggregated['Year'] = pd.DatetimeIndex(aggregated['Date']).year 
    aggregated['Month'] = pd.DatetimeIndex(aggregated['Date']).month
    return aggregated

def aggregate_test(data):
    data['Date'] = pd.to_datetime(data.Date, format='%Y-%m-%d')
    data['Year'] = pd.DatetimeIndex(data['Date']).year 
    data['Month'] = pd.DatetimeIndex(data['Date']).month
    return data

In [7]:
agg_train = aggregate_train(train)

In [8]:
agg_test = aggregate_test(test)

In [9]:
# there are no missing data in agg_train
agg_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4616 entries, 0 to 4615
Data columns (total 8 columns):
Date            4616 non-null datetime64[ns]
Trap            4616 non-null object
Latitude        4616 non-null float64
Longitude       4616 non-null float64
NumMosquitos    4616 non-null int64
WnvPresent      4616 non-null int64
Year            4616 non-null int64
Month           4616 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 288.6+ KB


In [12]:
agg_train.Year.value_counts()

2007    1459
2013    1163
2009    1006
2011     988
Name: Year, dtype: int64

In [ ]:
# there are no missing data in agg_test
agg_test.info()

In [13]:
agg_test.Year.value_counts()

2010    36557
2008    30498
2012    27115
2014    22123
Name: Year, dtype: int64

In [16]:
import math
# source: https://www.johndcook.com/blog/python_longitude_latitude/
def distance_on_unit_sphere(lat1, long1, lat2, long2):

    # Convert latitude and longitude to spherical coordinates in radians
    degrees_to_radians = math.pi/180.0

    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians

    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians

    # Compute spherical distance from spherical coordinates.

    # For two locations in spherical coordinates
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) =
    # sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length

    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) +
    math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )

    # Remember to multiply arc by the radius of the earth
    # in your favorite set of units to get length.
    return arc

In [17]:
# weather stations 1 and 2 lat/longitude
# Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
# Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level
station_1_lat = 41.995
station_1_lon = -87.933
station_2_lat = 41.786
station_2_lon = -87.752
# function to calculate whether station 1 or 2 (from weather.csv) is closer
def closest_station(lat, lon):
    if (distance_on_unit_sphere(lat, lon, station_1_lat, station_1_lon) <
        distance_on_unit_sphere(lat, lon, station_2_lat, station_2_lon)):
        return 1
    else: return 2

In [18]:
## REVISED 3/8/2019
# add station to indicate whether station 1 or 2 is closer
# create unique variable to combine train, weather and spray data
def station_var (data):
    data['Station'] = [closest_station(a,b) for (a, b) in zip(data.Latitude, data.Longitude)]
    data['unique'] =  data['Station'].astype(str) + str(": ") + data['Date'].astype(str)

In [19]:
station_var(agg_train)
station_var(agg_test)

###  WEATHER DATA

In [20]:
# Many numerical columns are imported as objects
weather.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
Depart          object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object

In [21]:
# Convert Date to date format
weather['Date'] = pd.to_datetime(weather['Date'], format='%Y-%m-%d')

In [22]:
# Calculate missing Tavg as average of Tmax and Tmin
weather['Tavg'] = weather.apply([lambda row: (row.Tmax + row.Tmin) / 2 if row.Tavg == 'M' else row.Tavg], axis=1)
weather['Tavg'] = pd.to_numeric(weather['Tavg'])

TypeError: ("'list' object is not callable", 'occurred at index 0')

In [ ]:
# Fill missing WetBulb from the other station
for k, v in enumerate(weather['WetBulb']):
    if v == 'M' and weather.iloc[k]['Station'] == 1:
        weather.loc[k,'WetBulb'] = weather['WetBulb'][k+1]
    if v == 'M' and weather.iloc[k]['Station'] == 2:
        weather.loc[k,'WetBulb'] = weather['WetBulb'][k-1]
weather['WetBulb'] = pd.to_numeric(weather['WetBulb'])

In [ ]:
# Engineer feature for daytime in minutes
sunset = pd.to_timedelta([i[:2] + ':' + i[2:] + ':00' for i in weather[weather['Station']==1]['Sunset']])
sunrise = pd.to_timedelta([i[:2] + ':' + i[2:] + ':00' for i in weather[weather['Station']==1]['Sunrise']])
daytime = sunset - sunrise
daytime_m = list([int(str(i).split(":")[0][-2:]) * 60 + int(str(i).split(":")[1]) for i in daytime])
weather['Daytime'] = list(sum((zip(daytime_m,daytime_m)),()))

In [23]:
# Fill trace values as 0.005
weather['PrecipTotal'].replace('  T', '0.005', inplace=True)
# Fill missing values from the other Station
for k, v in enumerate(weather['PrecipTotal']):
    if v == 'M' and weather.iloc[k]['Station'] == 1:
        weather.loc[k,'PrecipTotal'] = weather['PrecipTotal'][k+1]
    if v == 'M' and weather.iloc[k]['Station'] == 2:
        weather.loc[k,'PrecipTotal'] = weather['PrecipTotal'][k-1]
weather['PrecipTotal'] = pd.to_numeric(weather['PrecipTotal'])

In [24]:
# Fill missing values for Depart, Heat, Cool, AvgSpeed from the other station
cols = ['Depart', 'Heat', 'Cool', 'AvgSpeed']
for col in cols:
    for k, v in enumerate(weather[col]):
        if v == 'M' and weather.iloc[k]['Station'] == 1:
            weather.loc[k,col] = weather[col][k+1]
        if v == 'M' and weather.iloc[k]['Station'] == 2:
            weather.loc[k,col] = weather[col][k-1]
    weather[col] = pd.to_numeric(weather[col])

In [25]:
# Check column data types after cleaning
# We skipped cleaning for CodeSum, Depth, Water1, SnowFall, StnPressure, SeaLevel
# because we do not find them useful
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                   object
Depart                  int64
DewPoint                int64
WetBulb                object
Heat                    int64
Cool                    int64
Sunrise                object
Sunset                 object
CodeSum                object
Depth                  object
Water1                 object
SnowFall               object
PrecipTotal           float64
StnPressure            object
SeaLevel               object
ResultSpeed           float64
ResultDir               int64
AvgSpeed              float64
dtype: object

## Combine weather & train/test data for ease of analysis

In [26]:
# create unique variable to combine train and weather data
weather['unique'] = weather['Station'].astype(str) + str(": ") + weather['Date'].astype(str)

In [27]:
combined_train = pd.merge(agg_train, weather, how="inner", on="unique")
combined_test = pd.merge(agg_test, weather, how="inner", on="unique")

In [28]:
# dropping duplicate and unnecessary variables
combined_train = combined_train.drop(["Date_y", "Station_y", "unique"], axis=1)
combined_train.rename(columns={"Date_x": "Date"}, inplace=True)
combined_train.rename(columns={"Station_x": "Station"}, inplace=True)
combined_test = combined_test.drop(["Date_y", "Station_y", "unique"], axis=1)
combined_test.rename(columns={"Station_x": "Station"}, inplace=True)

# NEW 3/8/2019 - to create YYMM
# bin the surveillance efforts by month
combined_train ["YYMM"] = tuple(zip(combined_train.Year, combined_train.Month))
combined_test ["YYMM"] = tuple(zip(combined_test.Year, combined_test.Month))

In [29]:
combined_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4616 entries, 0 to 4615
Data columns (total 30 columns):
Date            4616 non-null datetime64[ns]
Trap            4616 non-null object
Latitude        4616 non-null float64
Longitude       4616 non-null float64
NumMosquitos    4616 non-null int64
WnvPresent      4616 non-null int64
Year            4616 non-null int64
Month           4616 non-null int64
Station         4616 non-null int64
Tmax            4616 non-null int64
Tmin            4616 non-null int64
Tavg            4616 non-null object
Depart          4616 non-null int64
DewPoint        4616 non-null int64
WetBulb         4616 non-null object
Heat            4616 non-null int64
Cool            4616 non-null int64
Sunrise         4616 non-null object
Sunset          4616 non-null object
CodeSum         4616 non-null object
Depth           4616 non-null object
Water1          4616 non-null object
SnowFall        4616 non-null object
PrecipTotal     4616 non-null float64
StnPr

In [30]:
# how many trap observations per month
combined_train["YYMM"].value_counts().sort_index()

(2007, 5)      18
(2007, 6)     120
(2007, 7)     260
(2007, 8)     574
(2007, 9)     382
(2007, 10)    105
(2009, 5)      41
(2009, 6)     255
(2009, 7)     285
(2009, 8)     180
(2009, 9)     208
(2009, 10)     37
(2011, 6)     193
(2011, 7)     263
(2011, 8)     252
(2011, 9)     280
(2013, 6)     248
(2013, 7)     285
(2013, 8)     357
(2013, 9)     273
Name: YYMM, dtype: int64

In [31]:
combined_train.Year.value_counts()

2007    1459
2013    1163
2009    1006
2011     988
Name: Year, dtype: int64

## Data Cleaning - Spray Data

In [32]:
#NEW 3/8/2019 - to create YYMM
spray['Date'] = pd.to_datetime(spray.Date, format='%Y-%m-%d')
spray['Year'] = pd.DatetimeIndex(spray['Date']).year 
spray['Month'] = pd.DatetimeIndex(spray['Date']).month
spray["YYMM"] = tuple(zip(spray.Year, spray.Month))
del spray['Time']

In [33]:
# there are no missing data in spray
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 6 columns):
Date         14835 non-null datetime64[ns]
Latitude     14835 non-null float64
Longitude    14835 non-null float64
Year         14835 non-null int64
Month        14835 non-null int64
YYMM         14835 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 695.5+ KB


In [35]:
spray.Year.value_counts()

2013    12626
2011     2209
Name: Year, dtype: int64

## EDA

#### Maps of WNV by year

In [ ]:
# source: https://www.kaggle.com/neilsummers/west-nile-heatmap-by-year
from sklearn.neighbors import KernelDensity

mapdata = plt.imread('../data/map1.png')
traps = pd.read_csv('../data/train.csv', parse_dates=['Date'])[['Date', 'Trap', 'Longitude', 'Latitude', 'WnvPresent']]

alpha_cm = plt.cm.Reds
alpha_cm._init()
alpha_cm._lut[:-3,-1] = np.linspace(0, 1, alpha_cm.N)
aspect = mapdata.shape[0] / mapdata.shape[1]
lon_lat_box = (-88, -87.5, 41.6, 42.1)

plt.figure(figsize=(18,24))
for year, subplot in zip([2007, 2009, 2011, 2013], [221, 222, 223, 224]):
    sightings = traps[(traps['WnvPresent'] > 0) & (traps['Date'].apply(lambda x: x.year) == year)]
    sightings = sightings.groupby(['Date', 'Trap', 'Longitude', 'Latitude']).max()['WnvPresent'].reset_index()
    X = sightings[['Longitude', 'Latitude']].values
    kd = KernelDensity(bandwidth=0.02)
    kd.fit(X)

    xv,yv = np.meshgrid(np.linspace(-88, -87.5, 100), np.linspace(41.6, 42.1, 100))
    gridpoints = np.array([xv.ravel(),yv.ravel()]).T
    zv = np.exp(kd.score_samples(gridpoints).reshape(100,100))
    plt.subplot(subplot)
    plt.gca().set_title(year)
    plt.imshow(mapdata,
               extent=lon_lat_box,
               aspect=aspect)
    plt.imshow(zv, 
               origin='lower',
               cmap=alpha_cm,
               extent=lon_lat_box,
               aspect=aspect)
    plt.tight_layout()
    locations = traps[['Longitude', 'Latitude']].drop_duplicates().values
    plt.scatter(locations[:,0], locations[:,1], c='blue', marker='x')
    plt.scatter([station_1_lon, station_2_lon], [station_1_lat,station_2_lat], c='purple', marker='o')

#### Maps of Spray activities by year

In [ ]:
mapdata = imread('../data/map2.png')

lon_lat_box = (-88.15, -87.5, 41.6, 42.45)

plt.figure(figsize=(18,14))
for year, subplot in zip([2011, 2013], [121, 122]):
    sightings = traps[(traps['WnvPresent'] > 0) & (traps['Date'].apply(lambda x: x.year) == year)]
    sightings = sightings.groupby(['Date', 'Trap','Longitude', 'Latitude']).max()['WnvPresent'].reset_index()
    X = sightings[['Longitude', 'Latitude']].values
    kd = KernelDensity(bandwidth=0.02)
    kd.fit(X)

    xv,yv = np.meshgrid(np.linspace(-88.15, -87.5, 100), np.linspace(41.6, 42.45, 100))
    gridpoints = np.array([xv.ravel(),yv.ravel()]).T
    zv = np.exp(kd.score_samples(gridpoints).reshape(100,100))
    plt.subplot(subplot)
    plt.gca().set_title(year)
    plt.imshow(mapdata,
               extent=lon_lat_box,
               aspect=aspect)
    plt.imshow(zv, 
               origin='lower',
               cmap=alpha_cm,
               extent=lon_lat_box,
               aspect=aspect)
    plt.tight_layout()
    locations_trap = traps[['Longitude', 'Latitude']].drop_duplicates().values
    plt.scatter(locations_trap[:,0], locations_trap[:,1], c='blue', marker='x')
    plt.scatter([station_1_lon, station_2_lon], [station_1_lat,station_2_lat], c='purple', marker='o')
    locations_spray = spray[['Longitude', 'Latitude']].drop_duplicates().values
    plt.scatter(locations_spray[:,0], locations_spray[:,1], c='orange', alpha=0.05)

#### Pairplot for weather data

In [ ]:
# look at relationships between features describing temperature
sns.pairplot(weather[['Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb']])

#### Visualizing the trap locations in comparison with the two weather stations

In [ ]:
# not necessary anymore
"""
# look at dispersion of traps geographically
plt.scatter(agg_train.Longitude, agg_train.Latitude, c = agg_train.Station)
plt.scatter([station_1_lon, station_2_lon], [station_1_lat,station_2_lat], c = 'r')
plt.show()
"""

#### Top 20 locations of highest number of mosquitoes

In [ ]:
## revised 5/8/2019
#top 20 locations of highest number of mosquitoes
lat_long = combined_train.groupby(["Latitude","Longitude"])['NumMosquitos'].agg(['mean','count'])
lat_long= lat_long.sort_values(by="mean", ascending=False).head(20)
#total_mosquitoes = total_mosquitoes.reset_index()
#sum = total no. of mosquitoes in the location
#count = no. of counts (by unique days)

lat_long

In [ ]:
## revised 5/8/2019
#period with highest number of mosquitoes 
total_mosquitoes = combined_train.groupby(["YYMM"])['NumMosquitos'].agg(['mean','count'])
total_mosquitoes= total_mosquitoes.sort_values(by="mean", ascending=False).head(20)
#total_mosquitoes = total_mosquitoes.reset_index()

total_mosquitoes

In [ ]:
## revised 5/8/2019
#period with highest number of WNV presents
total_WNV = combined_train.groupby(["YYMM"])['WnvPresent'].agg(['mean','count'])
total_WNV= total_WNV.sort_values(by="mean", ascending=False).head(20)

total_WNV

In [ ]:
## revised 5/8/2019
#traps corresponding to the locations of highest number of mosquitoes 
total_mosquitoes = combined_train.groupby(["Trap"])['NumMosquitos'].agg(['mean','count'])
total_mosquitoes= total_mosquitoes.sort_values(by="mean", ascending=False).head(20)
#total_mosquitoes = total_mosquitoes.reset_index()

total_mosquitoes

#### No. of Mosquitoes by Temp

In [ ]:
combined_train_ = combined_train[combined_train["WnvPresent"]== 1]
combined_train_

In [ ]:
temp = combined_train_.groupby(["Tavg","WetBulb","Year"])['NumMosquitos'].agg(['sum']).reset_index()
temp = pd.DataFrame(temp)
temp.sort_values("Tavg")
temp.to_csv(r'../data/temp.csv')

In [ ]:
plt.figure(figsize=(18,10))
plt.bar(temp['Tavg'],temp['sum'])
plt.title("Total number of mosquitoes with virus at different average temperatures")
plt.xlabel("Temperature")
plt.ylabel("No. of mosquitoes")

#### Traps corresponding to the locations of highest number of mosquitoes 

In [ ]:
#traps corresponding to the locations of highest number of mosquitoes 
total_mosquitoes = combined_train_.groupby(["Trap"])['NumMosquitos'].agg(['sum','count'])
total_mosquitoes= total_mosquitoes.sort_values(by="sum", ascending=False).head(20).reset_index()
#total_mosquitoes = total_mosquitoes.reset_index()

total_mosquitoes

In [ ]:
plt.figure(figsize=(18,10))
plt.bar(total_mosquitoes["Trap"],total_mosquitoes["sum"])
plt.title("Total number of mosquitoes in traps with virus")
plt.xlabel("Traps with virus")
plt.ylabel("No. of mosquitoes")

#### Grouping data into years as spray data contains only 2011 and 2013 data.

In [ ]:
list_var={}
def grouping_by_year(data):
    grps = data.groupby("Year")
    grps_key = list(grps.groups.keys())
    for i in range(len(list(grps.groups.keys()))):
        list_var["grouped_data_{}".format(grps_key[i])] = grps.get_group(grps_key[i])
    return list_var, grps_key

In [ ]:
grouping_by_year(combined_train)

In [ ]:
grouped_data_2007 = list_var["grouped_data_2007"]
grouped_data_2009 = list_var["grouped_data_2009"]
grouped_data_2011 = list_var["grouped_data_2011"]
grouped_data_2013 = list_var["grouped_data_2013"]

In [ ]:
list_var2={}
def grouping_by_year_spray(data):
    grps = data.groupby("Year")
    grps_key = list(grps.groups.keys())
    for i in range(len(list(grps.groups.keys()))):
        list_var2["grouped_data_spray{}".format(grps_key[i])] = grps.get_group(grps_key[i])
    return list_var2, grps_key

In [ ]:
grouping_by_year_spray(spray)

In [ ]:
grouped_data_spray_2011 = list_var2["grouped_data_spray2011"]
grouped_data_spray_2013 = list_var2["grouped_data_spray2013"]

#### Visualizing the trap locations with mosquitoes in comparison with the spray locations

In [ ]:
"""
# 2011
plt.scatter(grouped_data_spray_2011.Longitude, grouped_data_spray_2011.Latitude, c = 'r')
plt.scatter(grouped_data_2011.Longitude, grouped_data_2011.Latitude, c = grouped_data_2011.NumMosquitos)
plt.show()
"""

In [ ]:
"""
# 2013
plt.scatter(grouped_data_spray_2013.Longitude, grouped_data_spray_2013.Latitude, c = 'r')
plt.scatter(grouped_data_2013.Longitude, grouped_data_2013.Latitude, c = grouped_data_2013.NumMosquitos)
plt.show()
"""

In [ ]:
Nummos_bydate = combined_train.groupby(["Date"])['NumMosquitos'].agg(['sum']).reset_index()
Nummos_bydate = pd.DataFrame(Nummos_bydate )
Nummos_bydate .sort_values("Date")

In [ ]:
spray["spray"] = 1
Spray_bydate = spray.groupby(["Date"])["spray"].agg(['sum']).reset_index()
Spray_bydate = pd.DataFrame(Spray_bydate )
Spray_bydate .sort_values("Date")

In [ ]:
combined_spray_train = pd.merge(Spray_bydate, Nummos_bydate, how="outer", left_on="Date", right_on="Date")

In [ ]:
#clean the data frame for combined spray/train data
combined_spray_train.Date.fillna(combined_spray_train.Date, inplace=True)
del combined_spray_train['Date']
combined_spray_train.rename(columns={"sum_x": "Num_spray", "sum_y": "NumMosquitos"}, inplace=True)

In [ ]:
combined_spray_train

In [ ]:
# look at dispersion of virus incidence geographically
plt.scatter(agg_train.Longitude, agg_train.Latitude, c = agg_train.WnvPresent, alpha = .01)

In [ ]:
# look at incidence of cases over months
agg_train[['Month', 'NumMosquitos']].groupby("Month").sum().plot(kind='bar')

In [ ]:
# look at incidence of cases over months
agg_train[['Month', 'WnvPresent']].groupby("Month").sum().plot(kind='bar')

In [ ]:
# look at incidence of cases over years
agg_train[['Year', 'WnvPresent']].groupby('Year').sum().plot(kind='bar')

## Baseline

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, linear_model, datasets
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
import seaborn as sns



plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
X = combined_train.drop(["Latitude", "Longitude", "WnvPresent", "Trap", "Date", "YYMM", "Sunrise", "Sunset", "CodeSum", "Depth", "Water1", "SnowFall", "StnPressure", "SeaLevel"], axis=1)
y = combined_train.WnvPresent

In [ ]:
X.columns

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

In [ ]:
class model_evaluation:
    
    def __init__(self, y_test, predicted_value):
        self.y_test = y_test
        self.predicted_value = predicted_value
        
    def confusion_matrix (self):
        tn, fp, fn, tp = confusion_matrix(self.y_test, self.predicted_value).ravel()
        print("True Negatives: %s" % tn)
        print("False Positives: %s" % fp)
        print("False Negatives: %s" % fn)
        print("True Positives: %s" % tp)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        print("Precision: {}" .format (tp/(tp+fp)))
        print("Recall: {}" .format (tp/(tp+fn)))
        F1_score = 2*((precision*recall)/(precision+recall))
        print ("F1 score: {}" .format(2*((precision*recall)/(precision+recall))))
        self.score = F1_score

In [ ]:
lr_model = LogisticRegression()

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
predicted_lr = lr.predict(X_test)
lrg = model_evaluation (y_test,predicted_lr)
lrg.confusion_matrix ()
lr.score (X_test, y_test)

### MODELS

In [ ]:
#Test code, scaling of data, reduction of dimensionality, use of SVC.
from sklearn.decomposition import PCA
from sklearn.svm import SVC
pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=4)),
    ('clf', SVC(kernel = 'linear', C = 1))])
param_grid = dict(reduce_dims__n_components=[4,6,8],
                  clf__C=np.logspace(-4, 1, 6),
                  clf__kernel=['rbf','linear'])


grid = GridSearchCV(pipeline, cv=5, n_jobs=-1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
print(grid.score(X_test, y_test))


''' sample codes
#params={'model__C':[.01,.05,.1,.5,1,5,10],
           'model__penalty':['l1','l2']}
pipeline = Pipeline([
    ('vect', CountVectorizer(min_df=40,ngram_range=(1,4))),
    ('tfidf', TfidfTransformer()),
    ('model',LogisticRegression())])
grid = GridSearchCV(pipeline, cv=5, n_jobs=-1, param_grid=params ,scoring='roc_auc')
grid.fit(train['text'], train['output'])
grid.score(test['text'], test['output'])


#pipe = Pipeline([
        ('scale', StandardScaler()),
        ('reduce_dims', PCA(n_components=4)),
        ('clf', SVC(kernel = 'linear', C = 1))])

param_grid = dict(reduce_dims__n_components=[4,6,8],
                  clf__C=np.logspace(-4, 1, 6),
                  clf__kernel=['rbf','linear'])

grid = GridSearchCV(pipe, param_grid=param_grid, cv=3, n_jobs=1, verbose=2, scoring= 'accuracy')
grid.fit(X, y)
print(grid.best_score_)
print(grid.cv_results_)
'''

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', SVC(kernel = 'linear', C = 1))])
param_grid = dict(clf__C=np.logspace(-4, 1, 6),
                  clf__kernel=['rbf','linear'])


grid = GridSearchCV(pipeline, cv=3, n_jobs=-1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
#KNN
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=4)),
    ('clf', KNeighborsClassifier())])
param_grid = dict(reduce_dims__n_components=[4,6,8],
                  clf__n_neighbors=range(1, 201, 10),
                  clf__metric=['euclidean', 'manhattan'])


grid = GridSearchCV(pipeline, cv=3, n_jobs=-1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', KNeighborsClassifier())])
param_grid = dict(clf__n_neighbors=range(1, 201, 10),
                  clf__metric=['euclidean', 'manhattan'])


grid = GridSearchCV(pipeline, cv=3, n_jobs=-1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
#Multinomial is not applicable here due to the need for vectorizing, Decision trees will be used
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=4)),
    ('clf', DecisionTreeClassifier())])
param_grid = dict(reduce_dims__n_components=[4,6,8],
                  clf__max_depth=[None,1,2,3,4,5],
                  clf__max_features=['sqrt', 'log2', None],
                  clf__min_samples_split=[2,3,4,5,10,15,20,25,30,40,50])


grid = GridSearchCV(pipeline, cv=3, n_jobs=-1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)


'''
n_params = {
    'max_depth': [None,1,2,3,4,5],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': [2,3,4,5,10,15,20,25,30,40,50]
    }

grid3 = GridSearchCV(DecisionTreeClassifier(), n_params, cv=5)
grid3.fit(X,y)
print ('best score = {}'.format(grid3.best_score_))
grid3.best_params_
'''


In [ ]:
grid.best_params_

In [ ]:
list_2 = zip(list(X.columns), list(grid.best_estimator_.steps[2][1].feature_importances_))
pl = pd.Series(dict(list_2)).sort_values(ascending=False)
pl

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', DecisionTreeClassifier())])
param_grid = dict(clf__max_depth=[None,1,2,3,4,5],
                  clf__max_features=['sqrt', 'log2', None],
                  clf__min_samples_split=[2,3,4,5,10,15,20,25,30,40,50])


grid = GridSearchCV(pipeline, cv=5, n_jobs=-1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
print(grid.score(X_test, y_test))

In [ ]:
grid.best_params_

In [ ]:
#Example of feature importance: grid.best_estimator_.steps[1][1].feature_importances_
list_2 = zip(list(X.columns), list(grid.best_estimator_.steps[1][1].feature_importances_))
pl = pd.Series(dict(list_2)).sort_values(ascending=False)
pl.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
#Random forest- of course we need a random search first yeah?
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(200, 2000, 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
'''
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
'''

# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rc = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#rc_random = RandomizedSearchCV(estimator = rc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model


pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=4)),
    ('clf', RandomForestClassifier(random_state=42))])
param_grid = dict(reduce_dims__n_components=[4,6,8],
                  clf__n_estimators=[int(x) for x in np.linspace(200, 2000, 10)],
                  clf__max_features=['auto', 'sqrt'],
                  clf__max_depth=max_depth,
                  clf__min_samples_split=[2, 5, 10],
                  clf__min_samples_leaf=[1, 2, 4],
                  clf__bootstrap=[True, False]
                 )


rc_random = RandomizedSearchCV(pipeline, cv=3, n_jobs=-1, param_distributions=param_grid, n_iter = 100, verbose=2, scoring='roc_auc', random_state=42)
rc_random.fit(X_train, y_train)

In [ ]:
rc_random.best_params_

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=8)),
    ('clf', RandomForestClassifier(max_features='auto', bootstrap=True, min_samples_leaf=2,random_state=42))])

param_grid = dict(clf__n_estimators=[700, 800, 900],
                  clf__max_depth=[7,9,11,13],
                  clf__min_samples_split=[8,10,12,14],
                 )


grid = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
list_2 = zip(list(X.columns), list(grid.best_estimator_.steps[2][1].feature_importances_))
pl = pd.Series(dict(list_2)).sort_values(ascending=False)
pl.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(200, 2000, 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
'''
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
'''

# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rc = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#rc_random = RandomizedSearchCV(estimator = rc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model


pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=42))])
param_grid = dict(clf__n_estimators=[int(x) for x in np.linspace(200, 2000, 10)],
                  clf__max_features=['auto', 'sqrt'],
                  clf__max_depth=max_depth,
                  clf__min_samples_split=[2, 5, 10],
                  clf__min_samples_leaf=[1, 2, 4],
                  clf__bootstrap=[True, False]
                 )


rc_random = RandomizedSearchCV(pipeline, cv=3, n_jobs=-1, param_distributions=param_grid, n_iter = 100, verbose=2, scoring='roc_auc', random_state=42)
rc_random.fit(X_train, y_train)

In [ ]:
rc_random.best_params_

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', RandomForestClassifier(max_features='sqrt', bootstrap=True,random_state=42))])

param_grid = dict(clf__n_estimators=[1300, 1400, 1500],
                  clf__max_depth=[7,9,11,13],
                  clf__min_samples_split=[4,5,6],
                  clf__min_samples_leaf=[4,6,8]
                 )


grid = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
list_2 = zip(list(X.columns), list(grid.best_estimator_.steps[1][1].feature_importances_))
pl = pd.Series(dict(list_2)).sort_values(ascending=False)
pl.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=4)),
    ('clf', AdaBoostClassifier(random_state=42))])
param_grid = dict(reduce_dims__n_components=[4,6,8],
                  clf__n_estimators=[150, 200],
                  clf__learning_rate=[0.01,0.05,0.1,0.3,1],
                 )


ad_random = RandomizedSearchCV(pipeline, cv=3, n_jobs=-1, param_distributions=param_grid, n_iter = 100, verbose=2, scoring='roc_auc', random_state=42)
ad_random.fit(X_train, y_train)


In [ ]:
ad_random.best_params_

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=6)),
    ('clf', AdaBoostClassifier(random_state=42))])

param_grid = dict(clf__n_estimators=[200,250],
                  clf__learning_rate=[0.1,0.2,0.25,0.3,0.35]
                 )


grid = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
list_2 = zip(list(X.columns), list(grid.best_estimator_.steps[2][1].feature_importances_))
pl = pd.Series(dict(list_2)).sort_values(ascending=False)
pl.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
#Adaboost without PCA
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', AdaBoostClassifier(random_state=42))])
param_grid = dict(clf__n_estimators=[150, 200],
                  clf__learning_rate=[0.01,0.05,0.1,0.3,1],
                 )


ad_random = RandomizedSearchCV(pipeline, cv=3, n_jobs=-1, param_distributions=param_grid, n_iter = 100, verbose=2, scoring='roc_auc', random_state=42)
ad_random.fit(X_train, y_train)

In [ ]:
ad_random.best_params_

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', AdaBoostClassifier(random_state=42))])

param_grid = dict(clf__n_estimators=[200,250],
                  clf__learning_rate=[0.1,0.2,0.25,0.3,0.35]
                 )


grid = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
list_2 = zip(list(X.columns), list(grid.best_estimator_.steps[1][1].feature_importances_))
pl = pd.Series(dict(list_2)).sort_values(ascending=False)
pl.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
#Gradientboosting with PCA
from sklearn.ensemble import GradientBoostingClassifier
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=4)),
    ('clf', GradientBoostingClassifier(random_state=42))])
param_grid = dict(reduce_dims__n_components=[4,6,8],
                  clf__min_samples_split=[50, 100],
                  clf__min_samples_leaf=[1,2,4],
                  clf__max_features=['auto','sqrt', 'log2'],
                  clf__max_depth = [5,6,7,8]
                 )

gb_random = RandomizedSearchCV(pipeline, cv=3, n_jobs=-1, param_distributions=param_grid, n_iter = 100, verbose=2, scoring='roc_auc', random_state=42)
gb_random.fit(X_train, y_train)

In [ ]:
gb_random.best_params_

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('reduce_dims', PCA(n_components=8)),
    ('clf', GradientBoostingClassifier(min_samples_leaf=2, max_depth= 6,random_state=42))])

param_grid = dict(clf__min_samples_split=[100,150],
                  clf__max_features=['auto','sqrt', 'log2']
                 )


grid = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
list_2 = zip(list(X.columns), list(grid.best_estimator_.steps[2][1].feature_importances_))
pl = pd.Series(dict(list_2)).sort_values(ascending=False)
pl.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
#gradientboosting without PCA
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', GradientBoostingClassifier(random_state=42))])
param_grid = dict(clf__min_samples_split=[50, 100],
                  clf__min_samples_leaf=[1,2,4],
                  clf__max_features=['auto','sqrt', 'log2'],
                  clf__max_depth = [5,6,7,8]
                 )

gb_random = RandomizedSearchCV(pipeline, cv=3, n_jobs=-1, param_distributions=param_grid, n_iter = 100, verbose=2, scoring='roc_auc', random_state=42)
gb_random.fit(X_train, y_train)

In [ ]:
gb_random.best_params_

In [ ]:
pipeline = Pipeline([
    #('poly_feat', PolynomialFeatures(degree=3, interaction_only=True)),
    ('scale', StandardScaler()),
    ('clf', GradientBoostingClassifier(min_samples_leaf=1, max_features= 'sqrt',random_state=42))])

param_grid = dict(clf__min_samples_split=[30,40,50],
                  clf__max_depth=[3,4,5]
                 )


grid = GridSearchCV(pipeline, cv=3, n_jobs=1, param_grid=param_grid, verbose=2, scoring='roc_auc')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.cv_results_)
grid.score(X_test, y_test)

In [ ]:
grid.best_params_

In [ ]:
list_2 = zip(list(X.columns), list(grid.best_estimator_.steps[1][1].feature_importances_))
pl = pd.Series(dict(list_2)).sort_values(ascending=False)
pl.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

Some takeaways, random forest seems to do the best. PCA did not really help much and could be due to small number of features. Adaboost learning rate seems to balance off with the estimators. higher estimators = lower learning rate. When doing decision trees, features that are the most important is heavily skewed to number of mosquitos. Baseline should be treated as 94% because this is supposed to be rare. Considering a binary classification problem, one can always assume the mosquito is not infected and still get it right 94% of the time.

Moving forward: Makes sense to predict the number of mosquitos first? then do a classification? Read other solutions. Find ways to process the features, date/time stuff and encoding of some of the categorical ones. Break the baseline.

Done: Pipelines Gridsearch Randomized search PCA